In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import torch
import torchvision
import psycopg2
import random
import re
import time
import csv
import copy
import statsmodels
from functools import reduce 
from datetime import datetime as dt
import datetime
import shutil
import tempfile
from datetime import timedelta

from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data_utils
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision import models
from tempfile import TemporaryDirectory
from torchvision import transforms
from torchsample import transforms as ts_transforms
from matplotlib import pyplot as plt
from PIL import Image
from geopy.distance import vincenty 
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
from zipfile import ZipFile
import pygrib

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score

from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

# Connect to Database

## Configuration File

In [2]:
with open('../data/helpers/DBconfig.csv', mode='r') as infile:
    reader = csv.reader(infile)
    config = {rows[0]:rows[1] for rows in reader}

## Connection to DB

In [3]:
# Credentials fo AWS database
dsn_database = config['database']           # e.g. "compose"
dsn_hostname = "fogdb.cmse2tqlcvsl.eu-west-1.rds.amazonaws.com" # e.g.: "aws-us-east-1-portal.4.dblayer.com"
dsn_port = "9418"                 # e.g. 11101 
dsn_uid = config['\ufeffusername']       # e.g. "admin"
dsn_pwd = config ['password']     # e.g. "xxx"

try:
    conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
    print("Connecting to database:\n{}\n".format(conn_string))
    conn=psycopg2.connect(conn_string)
except:
    print ("\nNo connection to the database could be established.")

# Create cursor
cursor = conn.cursor()

# Show database tables
print('Tables in fog database:')
cursor.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
cursor.fetchall()

Connecting to database:
host=fogdb.cmse2tqlcvsl.eu-west-1.rds.amazonaws.com port=9418 dbname=FOGDB user=KNMIFOG password=fog&mist123

Tables in fog database:


[('cameras',),
 ('day_phases',),
 ('images',),
 ('manual_annotations',),
 ('locations',),
 ('image_features',),
 ('meteo_features_stations',),
 ('test_table_images',),
 ('meteo_stations',),
 ('meteo_features_copy',)]

# Acquire Target Data for Training (Subset)

## Distance DF

In [4]:
distance_df = pd.read_csv('../data/helpers/distanceKNMIStationsToLocations.csv')

## Get All KNMI Images Data

In [5]:
def get_path(filepath):
    '''
    Used for getting the right filepath.
    '''
    regex = re.compile(r'\d[/].*$')
    search = re.search(regex, filepath)
    jpg_name = search.group(0)[2:]

    # Second search for filepaths of highways
    regex2 = re.compile(r'[A]\d*-.*')
    search2 = re.search(regex2, jpg_name)
    
    if search2 != None:
        jpg_name = search2.group(0)
        return str(jpg_name)
    
    return jpg_name

# cursor.execute("SELECT * FROM images WHERE day_phase = '1'")
cursor.execute("SELECT * FROM images")

# Fetch
img_df = pd.DataFrame(cursor.fetchall(), columns=['img_id', 'camera_id', 'datetime', 'filepath', 'day_phase'])
img_df['filepath'] = img_df['filepath'].apply(get_path, 1)

## Bind Stations to Locations

In [6]:
# Fetch all the camera/location id pairs and put into df
cursor.execute("SELECT * FROM cameras")
df_cameras = pd.DataFrame(cursor.fetchall(), columns=['camera_id', 'location_id', 'cam_description', 'camera_name'])

# Merge image df with the cameras df and then with distance df
merged_image_cameras = pd.merge(img_df, df_cameras, on='camera_id')
merged_nearest = pd.merge(merged_image_cameras, distance_df, on='location_id')

## Get Visibility and Meteo Metrics

In [7]:
cursor.execute("SELECT * FROM meteo_features_copy")

df_meteo_features = pd.DataFrame(cursor.fetchall(), columns=['key','MeteoStationLocationID', 'datetime', 
                                                             'wind_speed', 'rel_humidity', 'air_temp', 'dew_point',
                                                            'mor_vis'])

## Add Missing Meteorological Data

### Windspeed Data

In [8]:
with open('../data/helpers/windspeed.csv', 'r') as file:
    csv_read = csv.reader(file)
    next(csv_read)
    
    c = 0
    for row in csv_read:
        if row[1] == '240_W_18Cm27':
            location_name = 'SCHIPHOL airport'
            meteo_id = 542
        elif row[1] == '344_W_06t':
            location_name = 'ROTTERDAM airport'
            meteo_id = 536
        elif row[1] == '380_W_04t':
            location_name = 'EELDE airport'
            meteo_id = 506
            
        date = row[0][:15].replace('_', '')
        
        if date[8:10] == '24':
            date = date[:8] + '00' + date[10:]

        datetime = dt.strptime(date, '%Y%m%d%H%M%S')
        indices = df_meteo_features[(df_meteo_features['datetime'] == datetime) & 
                          (df_meteo_features['MeteoStationLocationID'] == meteo_id)].index

        for idx in indices:
            try:
                c += 1
                df_meteo_features.at[idx, 'wind_speed'] = row[2]
            except:
                print(location_name, datetime)
                
            if c % 10000 == 0:
                print('Added {} wind speed values'.format(c))         

Added 10000 wind speed values
EELDE airport 2017-07-08 14:10:00
EELDE airport 2017-07-08 14:20:00
ROTTERDAM airport 2017-07-08 14:30:00
EELDE airport 2017-07-08 14:30:00
SCHIPHOL airport 2017-07-08 14:40:00
ROTTERDAM airport 2017-07-08 14:40:00
EELDE airport 2017-07-08 14:40:00
SCHIPHOL airport 2017-07-08 14:50:00
ROTTERDAM airport 2017-07-08 14:50:00
EELDE airport 2017-07-08 14:50:00
ROTTERDAM airport 2017-07-08 15:00:00
EELDE airport 2017-07-08 15:00:00
SCHIPHOL airport 2017-07-08 15:10:00
ROTTERDAM airport 2017-07-08 15:10:00
EELDE airport 2017-07-08 15:10:00
SCHIPHOL airport 2017-07-08 15:20:00
ROTTERDAM airport 2017-07-08 15:20:00
EELDE airport 2017-07-08 15:20:00
SCHIPHOL airport 2017-07-08 15:30:00
ROTTERDAM airport 2017-07-08 15:30:00
SCHIPHOL airport 2017-07-08 15:40:00
ROTTERDAM airport 2017-07-08 15:40:00
SCHIPHOL airport 2017-07-08 15:50:00
ROTTERDAM airport 2017-07-08 15:50:00
EELDE airport 2017-07-08 15:50:00
ROTTERDAM airport 2017-07-08 16:00:00
SCHIPHOL airport 2017-07-

### Temperature/Humidity Data

In [10]:
with open('../data/helpers/temp_humidity.csv', 'r') as file:
    csv_read = csv.reader(file)
    next(csv_read)
    c = 0
    for row in csv_read:
        date = row[0][:15].replace('_', '')
        air_temp = row[2]
        rel_hum = row[4]
        meteo_id = 542
        
        if date[8:10] == '24':
            date = date[:8] + '00' + date[10:]
            
        datetime = dt.strptime(date, '%Y%m%d%H%M%S')
        
        indices = df_meteo_features[(df_meteo_features['datetime'] == datetime) & 
                                    (df_meteo_features['MeteoStationLocationID'] == meteo_id)].index

        for idx in indices:

            try:
                c += 1
                df_meteo_features.at[idx, 'air_temp'] = air_temp
                df_meteo_features.at[idx, 'rel_humidity'] = rel_hum
            except:
                continue
                
            if c % 10000 == 0:
                print('Added {} temperature and humidity values'.format(c))    

Added 10000 temperature and humidity values
Added 20000 temperature and humidity values
Added 30000 temperature and humidity values


## Bind Images to Visibility

In [12]:
loc_names = ['De Bilt (260_A_a)', 'Cabauw (348)', 'BEEK airport', 'EELDE airport', 'ROTTERDAM airport', 'SCHIPHOL airport']

# Categorise images
def ordinal_visibility(mor_vis):
    if mor_vis > 1000:
        return 0
    elif mor_vis < 250:
        return 2
    elif mor_vis >= 250 and mor_vis <= 1000:
        return 1
    
# Here the meteo features of closest meteo station are linked to every camera
main_df = pd.merge(merged_nearest, df_meteo_features, on=['MeteoStationLocationID', 'datetime'])
main_df['visibility'] = main_df['mor_vis'].apply(ordinal_visibility, 1)

# Location to main df
cursor.execute("SELECT * FROM locations")
locations_df = pd.DataFrame(cursor.fetchall(), columns=['location_id', 'location_name', 'long', 'lat'])
main_df = pd.merge(locations_df, main_df, on='location_id')

# Make sure that meteo variables above 7500m distance are nan. We want to fill these with kriging
main_df.loc[~main_df.location_name.isin(loc_names), ['wind_speed', 'rel_humidity', 'air_temp', 'dew_point']] = np.nan

# # Distribution of labels
main_df['visibility'].value_counts()

0.0    3143058
1.0      57997
2.0      45916
Name: visibility, dtype: int64

## Bind Location Names to Location IDs

In [15]:
# Drop unnecessary columns
retain_columns = ['location_id', 'long', 'lat', 'location_name', 'camera_id', 'datetime', 'filepath', 'MeteoStationLocationID', 
                 'camera_name', 'meteo_station_id', 'meteo_station_name', 'distanceInMeters', 'wind_speed',
                 'rel_humidity', 'air_temp', 'dew_point', 'mor_vis', 'visibility']

main_df = main_df[retain_columns]

KeyError: "['long' 'lat' 'location_name'] not in index"

### Change the File Path for Certain  Pictures

In [13]:
def change_filepaths(filepath):
    '''
    Changes the filepaths of old de Bilt images and of Cabauw images.
    
    :param filepath: Either a de Bilt or Cabauw image filepath.
    '''
    if 'nobackup/users/' in filepath:
        return filepath[-29:]
    elif 'CABAUW' in filepath:
        return filepath[7:]
    else:
        return filepath
    
def change_cabauw(filepath):
    if 'CABAUW' in filepath:
        return filepath[7:]
    else:
        return filepath

main_df['filepath'] = main_df['filepath'].apply(change_filepaths, 1)

### Exclude NaN's

In [8]:
main_df = main_df[main_df['visibility'].notnull()]
main_df = main_df[main_df['rel_humidity'].notnull()]
main_df = main_df[main_df['wind_speed'].notnull()]
main_df= main_df[main_df['air_temp'].notnull()]

In [16]:
def get_zipfile_path(date):

    original_hour = date[-2:]
    
    # Change the hour for getting right shit
    if int(original_hour) < 7:
        hour = '00'
    elif 6 < int(original_hour) < 13:
        hour = '06'
    elif 12 < int(original_hour) < 19:
        hour = '12'
    elif 18 < int(original_hour) < 24:
        hour = '18'
        
    difference = int(original_hour) - int(hour)

    if difference >= 10:
        grib_hour = str(difference)
    else:
        grib_hour = '0{}'.format(str(difference))

    zip_date = date[:-2]+hour
    
    # Get path to zipfile
    zipfile_name = 'HARM_{}_P1.zip'.format(zip_date)
    zip_path = '/Volumes/externe schijf/timWeatherModel/{}'.format(zipfile_name)
    
    return zip_path, zip_date, grib_hour, original_hour

def open_grib(zip_path, zip_date, grib_hour):
    
    # Create temporary directory
    tDir = tempfile.mkdtemp('Harmonies')
    
    with TemporaryDirectory() as tmp_dir:
        
        # Open zipfile and get grib filepath
        archive = ZipFile(zip_path)

        grib_name = 'HA36_P1_{}00_0{}00_GB'.format(zip_date, grib_hour)

        # Extract right grib file from zip and bring to temporary directory
        extract_zip = archive.extract(grib_name, tmp_dir)
        gribfile = pygrib.open('{}/{}'.format(tmp_dir, grib_name))
        
    return gribfile

def get_latlon_idx(lats, lons, obs_lat, obs_lon):
    # Get the absolute distances between all grid points and observation point
    abslat = np.abs(lats - obs_lat)
    abslon = np.abs(lons - obs_lon)

    # Get the absolute distance for 
    lat_index = np.argmin(abslat)
    lon_index = np.argmin(abslon)

    c = np.maximum(abslon, abslat)
    latlon_idx = np.argmin(c)
    
    return latlon_idx

def calculate_dewpoint(RH, T):
    '''
    Calculates dewpoint give relative humidity and temperature. 0.08 is added at the end, because otherwise
    calculations would be inconsistent with dataframe values.
    '''
    RH = RH * 100
    DP = 243.04*(np.log(RH/100)+((17.625*T)/(243.04+T)))/(17.625-np.log(RH/100)-((17.625*T)/(243.04+T))) + 0.08
    return DP

def get_meteo_grids(grib):

    # Get variable values for date
    temps = grib.select(name='2 metre temperature')[0]
    rel_hums = grib.select(name='Relative humidity')[0]
    
    # Calculate the wind speed
    windcomponent_U = grib.select(name='10 metre U wind component')[0].values
    windcomponent_V = grib.select(name='10 metre V wind component')[0].values
    windspeeds = np.sqrt(windcomponent_U ** 2 + windcomponent_V **2)

    # Latitudes and longitudes are the same for every variable, because same grid is used
    lats, lons = temps.latlons()

    temps, rel_hums = temps.values, rel_hums.values
    
    return lats, lons, temps, rel_hums, windspeeds

In [37]:
def fill_harmonie(main_df):

    unique_dates = main_df['datetime'].unique()
    num_dates = len(unique_dates)

    for c, one_date in enumerate(unique_dates):

        # Get df for unique date and extract date
        date_df = main_df[main_df['datetime'] == one_date]

        # Convert date to string for getting zipfile
        t = pd.to_datetime(str(one_date)) 
        timestring = t.strftime('%Y%m%d%H')

        # Get the zipfile path and the right date for getting gribfile in ZIP
        zip_path, zip_date, grib_hour, or_hour = get_zipfile_path(timestring)
        
        try:
            grib = open_grib(zip_path, zip_date, grib_hour)
            
            # Obtain grid point positions and meteo variables
            lats, lons, temps, rel_hums, windspeeds = get_meteo_grids(grib)
            for idx, row in date_df.iterrows():

                # Get the closest lat/lon index of grid
                obs_lat, obs_lon = row['lat'], row['long']
                closest_idx = get_latlon_idx(lats, lons, obs_lat, obs_lon)

                # Get the variable values
                temp, rel_hum, windspeed = temps.flat[closest_idx], rel_hums.flat[closest_idx], windspeeds.flat[closest_idx]
                temp = float(temp) - 272.15

                dew_point = calculate_dewpoint(rel_hum, temp)

                if np.isnan(row['air_temp']):
                    main_df.at[idx, 'air_temp'] = temp
                if np.isnan(row['rel_humidity']):
                    main_df.at[idx, 'rel_humidity'] = rel_hum
                if np.isnan(row['wind_speed']):
                    main_df.at[idx, 'wind_speed'] = windspeed
                if np.isnan(row['dew_point']):
                    main_df.at[idx, 'dew_point'] = dew_point

        except:
            print('Grib-file unavailable at index: {}, for date: {}'.format(c, timestring))
            print(zip_path, zip_date)
            print(timestring)
            print(date_df['filepath'])
            continue
    
    
        if c % 500 == 0:
            print('Iterated over {} of {} unique dates'.format(c, num_dates))
    
    return main_df

In [10]:
main_df = fill_harmonie(main_df)

NameError: name 'main_df' is not defined

## DF for Filling Test Files

In [38]:
def round_minutes(tm):
    tm = tm - timedelta(minutes=tm.minute % 10,
                                 seconds=tm.second,
                                 microseconds=tm.microsecond)
    return tm

with open('/Volumes/TIMKNMI/KNMIPictures/RWS/TestImagesRefined.txt') as filestream:
    test_filenames = []
    c = 0
    for row in filestream:
        row = row.strip().split(',')
        filename = row[0]
        test_filenames.append(filename)
        c+=1
        
test_loc_df = pd.merge(merged_nearest, locations_df, on='location_id')
test_loc_df = test_loc_df[test_loc_df['filepath'].isin(test_filenames)]  
test_loc_df['datetime'] = test_loc_df['datetime'].apply(round_minutes, 1)
test_loc_df['wind_speed'], test_loc_df['rel_humidity'], test_loc_df['air_temp'], test_loc_df['dew_point'] = np.nan, np.nan, np.nan, np.nan

In [39]:
test_df = fill_harmonie(test_loc_df)

Iterated over 0 of 450 unique dates
Grib-file unavailable at index: 32, for date: 2018040417
/Volumes/externe schijf/timWeatherModel/HARM_2018040412_P1.zip 2018040412
2018040417
152723    A2-HM770-ID10914_20180404_1740.jpg
Name: filepath, dtype: object
Grib-file unavailable at index: 46, for date: 2017070514
/Volumes/externe schijf/timWeatherModel/HARM_2017070512_P1.zip 2017070512
2017070514
227470    A2-HM776-ID10915_20170705_1402.jpg
Name: filepath, dtype: object
Grib-file unavailable at index: 104, for date: 2017111013
/Volumes/externe schijf/timWeatherModel/HARM_2017111012_P1.zip 2017111012
2017111013
410587    A2-HM894-ID13471_20171110_1320.jpg
Name: filepath, dtype: object
Grib-file unavailable at index: 116, for date: 2017121214
/Volumes/externe schijf/timWeatherModel/HARM_2017121212_P1.zip 2017121212
2017121214
462417    A2-HM905-ID13470_20171212_1450.jpg
Name: filepath, dtype: object
Grib-file unavailable at index: 399, for date: 2017070510
/Volumes/externe schijf/timWeatherMo

### Pickle or Unpickle the Main Dataframe

In [10]:
q1 = input('do you want to pickle a new main_df dataframe? (y/n)')

if q1 == 'y':
    main_df.to_pickle('../data/semi-processed/harmonie_filled_well')
    print('succesfully pickled DF')
else:
    q2 = input('do you want to unpickle the current saved dataframe? (y/n)')
    if q2 == 'y':
        main_df = pd.read_pickle('../data/semi-processed/before_filling_meteo')
        print('succesfully unpickled DF')

do you want to pickle a new main_df dataframe? (y/n)y
succesfully pickled DF


In [64]:
main_df = pd.read_pickle('../data/semi-processed/harmonie_filled_well')

In [65]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100790 entries, 0 to 1326214
Data columns (total 18 columns):
location_id               1100790 non-null int64
long                      1100790 non-null float64
lat                       1100790 non-null float64
location_name             1100790 non-null object
camera_id                 1100790 non-null int64
datetime                  1100790 non-null datetime64[ns]
filepath                  1100790 non-null object
MeteoStationLocationID    1100790 non-null int64
camera_name               1100790 non-null object
meteo_station_id          1100790 non-null int64
meteo_station_name        1100790 non-null object
distanceInMeters          1100790 non-null float64
wind_speed                1100790 non-null float64
rel_humidity              1100790 non-null float64
air_temp                  1100790 non-null float64
dew_point                 1100790 non-null float64
mor_vis                   1100790 non-null float64
visibility               

In [ ]:
KNMI_names = ['De Bilt (260_A_a)', 'Cabauw (348)', 'BEEK airport', 'EELDE airport', 'ROTTERDAM airport', 'SCHIPHOL airport']
meteo_variables = ['wind_speed', 'air_temp', 'rel_humidity', 'dew_point']

def distribution_meteo_variables(main_df):
    clip = {'wind_speed': (0, 12), 'air_temp': (-1000,1000), 'rel_humidity': (90, 120), 'dew_point':(-1000, 1000)}
    
    plot_df = main_df[main_df['location_name'].isin(KNMI_names)]
    fig = plt.figure(figsize=(15, 10))
    fig.tight_layout()
    
    # Boxplot for every variable
    for i, variable in enumerate(meteo_variables):
        
        if variable == 'rel_humidity':
            plot_df = plot_df[plot_df[variable] > 95]
    
        target_0 = plot_df.loc[plot_df['visibility'] == 0]
        target_1 = plot_df.loc[plot_df['visibility'] == 1]
        target_2 = plot_df.loc[plot_df['visibility'] == 2]
        
        # Make-up
        fig.add_subplot(2, 2, i + 1)
        plt.title('Density plot of {}'.format(variable))
        plt.ylabel('Density')
        
        sns.distplot(target_0[variable], hist=False, kde=True, kde_kws = {'linewidth':3, 'clip': clip[variable], 'shade':True}, label='no fog')
        sns.distplot(target_1[variable], hist=False, kde=True, kde_kws = {'linewidth':3,'clip': clip[variable], 'shade':True}, label='light fog')
        sns.distplot(target_2[variable], hist=False, kde=True, kde_kws = {'linewidth':3,'clip': clip[variable], 'shade':True},label='dense fog')
    
    plt.show()

## Close Database Connection

In [ ]:
conn.close()

## Image Parameters

In [55]:
IMG_SIZE = 100
CHANNELS = 3
test_df1 = test_df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 31218 to 8176156
Data columns (total 20 columns):
img_id                    804 non-null int64
camera_id                 804 non-null int64
datetime                  804 non-null datetime64[ns]
filepath                  804 non-null object
day_phase                 804 non-null int64
location_id               804 non-null int64
cam_description           804 non-null object
camera_name               804 non-null object
Unnamed: 0                804 non-null int64
meteo_station_id          804 non-null int64
meteo_station_name        804 non-null object
MeteoStationLocationID    804 non-null int64
distanceInMeters          804 non-null float64
location_name             804 non-null object
long                      804 non-null float64
lat                       804 non-null float64
wind_speed                797 non-null float64
rel_humidity              797 non-null float64
air_temp                  797 non-null float64
dew_po

## Load and Manipulate Test Data (Manually Labeled)

### Data Loading

In [57]:
LABELED_DIR = '/Volumes/TIMKNMI/KNMIPictures/RWS/'

# Used for storing filename:label pairs
image_list_test = []
target_list_test = []
filepath_list_test = []
meteo_list_test = []

# This opens all labeled files and finds the corresponding pictures and labels
with open(LABELED_DIR + 'TestImagesRefined.txt') as filestream:
    c = 0
    for row in filestream:
        row = row.strip().split(',')
        filename = row[0]
        label = row[1]
        label = label[0][0]
        
        datapoint_df = test_df1[test_df1['filepath'] == filename]
        meteo = datapoint_df[['wind_speed', 'rel_humidity', 'air_temp', 'dew_point']]
        
        if len(meteo) > 0:
            
            # Regex necessary elements
            highway = re.search(r'A\d*', filename).group(0)
            ID = re.search(r'ID\d*', filename).group(0)
            HM = re.search(r'HM\d*', filename).group(0)
            year_month = re.search(r'_\d*_', filename).group(0)[1:7]

            path = '{}{}/{}/{}/{}/{}'.format(LABELED_DIR, highway, HM, ID, year_month, filename)

            img = Image.open(path)
            img = img.resize((IMG_SIZE, IMG_SIZE))
            image_list_test.append(np.asarray(img))
            target_list_test.append(np.asarray(int(label)))
            filepath_list_test.append(np.asarray(path))
            meteo_list_test.append(np.asarray(meteo)[0])

            img.close()
        
        else:
            print(filename)
            print(len(np.asarray(meteo)))
            print(meteo)
            

test_features, test_targets, test_filepaths, test_meteo = np.asarray(image_list_test), np.asarray(target_list_test), np.asarray(filepath_list_test), np.asarray(meteo_list_test)
test_meteo = test_meteo.reshape(len(test_meteo), 4)

A2-HM742-ID10906_20171008_1531.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A2-HM770-ID10914_20180404_1740.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A2-HM776-ID10915_20170705_1402.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A2-HM894-ID13471_20171110_1320.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A2-HM905-ID13470_20171212_1450.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A15-HM397-ID106985_20170705_1042.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []
A15-HM786-ID12041_20180101_1040.jpg
0
Empty DataFrame
Columns: [wind_speed, rel_humidity, air_temp, dew_point]
Index: []


In [58]:
len(test_meteo)

797

In [ ]:
light = np.where(test_targets == 1)[0]
dense = np.where(test_targets == 2)[0]

for idx in light:
    print(test_filepaths[idx])
    img = test_features[idx]
    plt.imshow(img)
    plt.show()

## Load KNMI Train Data to Numpy Arrays

### Get Numpy Arrays

In [ ]:
KNMI_DIR = '/Volumes/TIMPP/KNMIPictures/'

image_list = []
target_list = []
filepath_list = []
meteo_list = []

KNMI_names = ['De Bilt (260_A_a)', 'Cabauw (348)', 'BEEK airport', 'EELDE airport', 'ROTTERDAM airport', 'SCHIPHOL airport']
KNMI_df = main_df[main_df['location_name'].isin(KNMI_names)]

for c, (index, row) in enumerate(KNMI_df.iterrows()):
    folder = row['filepath'].split('-')[0]
    folder = folder.split('_')[0]
    camera = row['camera_name']
    year_month = row['datetime'].strftime("%Y%m")  
    file = row['filepath']
    meteo = row[['wind_speed', 'rel_humidity', 'air_temp', 'dew_point']]
    
    filepath = '/Volumes/TIMPP/KNMIPictures/{}/{}/{}/{}'.format(folder, camera, year_month, file)

    try:
        img = Image.open(filepath)
        img = img.resize((IMG_SIZE, IMG_SIZE))
        image_list.append(np.asarray(img))
        img.close()

        # Get target, append to target list
        target = main_df[main_df['filepath'] == file]['visibility']
        target_list.append(target.iloc[0])
        filepath_list.append(filepath)
        meteo_list.append(meteo)

    # Catch erroneous files and continue if caught
    except:
        print('Could not load image: {}'.format(file))
        continue
    
    # Print number of loaded images every 500 iterations
    if c % 500 == 0:
        print('Number of df rows iterated: {} of {}'.format(c, len(KNMI_df)))

KNMI_images, KNMI_targets, KNMI_filepaths, KNMI_meteo = np.asarray(image_list), np.asarray(target_list), np.asarray(filepath_list), np.asarray(meteo_list)    

### Ommit Labels and Change Labels KNMI 

In [ ]:
# Get indices to delete from KNMI numpy arrays
del_knmi_idx = [i for i, v in enumerate(KNMI_filepaths) if 'BSRN-1' in v or 'Meetterrein_201606' in v or 'Meetterrein_201607' in v
            or 'Meetterrein_201608' in v]

# Ommit bad images from numpy arrays
KNMI_targets = np.delete(KNMI_targets, del_knmi_idx, 0)
KNMI_images = np.delete(KNMI_images, del_knmi_idx, 0)
KNMI_filepaths = np.delete(KNMI_filepaths, del_knmi_idx, 0)
KNMI_meteo = np.delete(KNMI_meteo, del_knmi_idx, 0)

# Change targets of wrongly labeled images
with open('../data/helpers/knmichangelabels') as file:
    for row in file:
        path, target = row.split(',')
        path_idx = np.where(KNMI_filepaths == path.strip("'"))
        KNMI_targets[path_idx] = target

In [ ]:
KNMI_targets = np.load('../data/processed/KNMI_train_targets.npy')
KNMI_images = np.load('../data/processed/KNMI_train_images.npy')
KNMI_filepaths = np.load('../data/processed/KNMI_train_filepaths.npy')
KNMI_meteo = np.load('../data/processed/KNMI_train_meteo.npy')

## Load Highway Train Data to Numpy Arrays

### Create Highway DF

In [12]:
schiphol_df = main_df[main_df['MeteoStationLocationID'] == 542]
schiphol_highways = schiphol_df[schiphol_df['location_name']!= 'SCHIPHOL airport']
eelde_df = main_df[main_df['MeteoStationLocationID'] == 506]
eelde_highways = eelde_df[eelde_df['location_name'] != 'EELDE airport']
highway_df = pd.concat([eelde_highways, schiphol_highways])
highway_df = highway_df[highway_df['distanceInMeters'] < 7500]

### Transform Data to Numpy Arrays

In [15]:
image_list = []
target_list = []
filepath_list = []
meteo_list = []
count = 1

for index, row in highway_df.iterrows():
    location = row['location_name'].split('-')
    year_month = row['datetime'].strftime("%Y%m")
    file = row['filepath']
    meteo = row[['wind_speed', 'rel_humidity', 'air_temp', 'dew_point']]
    filepath = '/Volumes/TIMKNMI/KNMIPictures/UnlabeledRWS/{}/{}/{}/{}/{}'.format(location[0], location[1], location[2],
                                                                                 year_month, file)
    try:
        img = Image.open(filepath)
        img = img.resize((IMG_SIZE, IMG_SIZE))
        image_list.append(np.asarray(img))
        img.close()

        # Get target, append to target list
        target = main_df[main_df['filepath'] == file]['visibility']
        target_list.append(target.iloc[0])
        filepath_list.append(filepath)
        meteo_list.append(meteo)
        count += 1

        # Print number of loaded images every 500 iterations
        if count % 500 == 0:
            print('Number of images loaded: {}'.format(count))

    #Catch erroneous files and continue if caught
    except:
        print('Could not load image: {}'.format(file))
        continue
    
highway_images, highway_targets, highway_filepaths, highway_meteo = np.asarray(image_list), np.asarray(target_list), np.asarray(filepath_list), np.asarray(meteo_list)

Number of images loaded: 500
Could not load image: A28-HM1893-ID11820_20180214_1200.jpg
Number of images loaded: 1000
Number of images loaded: 1500
Number of images loaded: 2000
Number of images loaded: 2500
Number of images loaded: 3000
Number of images loaded: 3500
Number of images loaded: 4000
Number of images loaded: 4500
Could not load image: A28-HM1893-ID11820_20180416_1830.jpg
Could not load image: A28-HM1893-ID11820_20180417_0510.jpg
Could not load image: A28-HM1893-ID11820_20180417_0530.jpg
Could not load image: A28-HM1893-ID11820_20180417_0540.jpg
Could not load image: A28-HM1893-ID11820_20180417_0610.jpg
Could not load image: A28-HM1893-ID11820_20180417_0650.jpg
Could not load image: A28-HM1893-ID11820_20180417_0910.jpg
Could not load image: A28-HM1893-ID11820_20180417_0920.jpg
Could not load image: A28-HM1893-ID11820_20180417_0930.jpg
Could not load image: A28-HM1893-ID11820_20180417_1000.jpg
Could not load image: A28-HM1893-ID11820_20180417_1010.jpg
Could not load image: A

Could not load image: A28-HM1893-ID11820_20180423_1640.jpg
Could not load image: A28-HM1893-ID11820_20180423_1650.jpg
Could not load image: A28-HM1893-ID11820_20180423_1700.jpg
Could not load image: A28-HM1893-ID11820_20180423_1720.jpg
Could not load image: A28-HM1893-ID11820_20180423_1730.jpg
Could not load image: A28-HM1893-ID11820_20180423_1740.jpg
Could not load image: A28-HM1893-ID11820_20180423_1750.jpg
Could not load image: A28-HM1893-ID11820_20180423_1800.jpg
Could not load image: A28-HM1893-ID11820_20180423_1810.jpg
Could not load image: A28-HM1893-ID11820_20180423_1830.jpg
Could not load image: A28-HM1893-ID11820_20180423_1840.jpg
Could not load image: A28-HM1893-ID11820_20180424_0420.jpg
Could not load image: A28-HM1893-ID11820_20180424_0430.jpg
Could not load image: A28-HM1893-ID11820_20180424_0440.jpg
Could not load image: A28-HM1893-ID11820_20180424_0450.jpg
Could not load image: A28-HM1893-ID11820_20180424_0500.jpg
Could not load image: A28-HM1893-ID11820_20180424_0510.j

Could not load image: A28-HM1893-ID11820_20180428_1440.jpg
Could not load image: A28-HM1893-ID11820_20180428_1450.jpg
Could not load image: A28-HM1893-ID11820_20180428_1500.jpg
Could not load image: A28-HM1893-ID11820_20180428_1510.jpg
Could not load image: A28-HM1893-ID11820_20180428_1520.jpg
Could not load image: A28-HM1893-ID11820_20180428_1530.jpg
Could not load image: A28-HM1893-ID11820_20180428_1540.jpg
Could not load image: A28-HM1893-ID11820_20180428_1550.jpg
Could not load image: A28-HM1893-ID11820_20180428_1600.jpg
Could not load image: A28-HM1893-ID11820_20180428_1610.jpg
Could not load image: A28-HM1893-ID11820_20180428_1630.jpg
Could not load image: A28-HM1893-ID11820_20180428_1640.jpg
Could not load image: A28-HM1893-ID11820_20180428_1650.jpg
Could not load image: A28-HM1893-ID11820_20180428_1700.jpg
Could not load image: A28-HM1893-ID11820_20180428_1710.jpg
Could not load image: A28-HM1893-ID11820_20180428_1720.jpg
Could not load image: A28-HM1893-ID11820_20180428_1730.j

Number of images loaded: 5000
Number of images loaded: 5500
Number of images loaded: 6000
Number of images loaded: 6500
Number of images loaded: 7000
Number of images loaded: 7500
Number of images loaded: 8000
Number of images loaded: 8500
Number of images loaded: 9000
Number of images loaded: 9500
Number of images loaded: 10000
Number of images loaded: 10500
Number of images loaded: 11000
Number of images loaded: 11500
Number of images loaded: 12000
Could not load image: A28-HM1907-ID11819_20180417_0510.jpg
Could not load image: A28-HM1907-ID11819_20180417_0530.jpg
Could not load image: A28-HM1907-ID11819_20180417_0540.jpg
Could not load image: A28-HM1907-ID11819_20180417_0610.jpg
Could not load image: A28-HM1907-ID11819_20180417_0910.jpg
Could not load image: A28-HM1907-ID11819_20180417_0930.jpg
Could not load image: A28-HM1907-ID11819_20180417_1000.jpg
Could not load image: A28-HM1907-ID11819_20180417_1010.jpg
Could not load image: A28-HM1907-ID11819_20180417_1020.jpg
Could not load

Could not load image: A28-HM1907-ID11819_20180422_1050.jpg
Could not load image: A28-HM1907-ID11819_20180422_1240.jpg
Could not load image: A28-HM1907-ID11819_20180422_1250.jpg
Could not load image: A28-HM1907-ID11819_20180422_1300.jpg
Could not load image: A28-HM1907-ID11819_20180422_1310.jpg
Could not load image: A28-HM1907-ID11819_20180422_1350.jpg
Could not load image: A28-HM1907-ID11819_20180422_1400.jpg
Could not load image: A28-HM1907-ID11819_20180422_1430.jpg
Could not load image: A28-HM1907-ID11819_20180422_1450.jpg
Could not load image: A28-HM1907-ID11819_20180422_1500.jpg
Could not load image: A28-HM1907-ID11819_20180422_1510.jpg
Could not load image: A28-HM1907-ID11819_20180422_1550.jpg
Could not load image: A28-HM1907-ID11819_20180422_1600.jpg
Could not load image: A28-HM1907-ID11819_20180422_1620.jpg
Could not load image: A28-HM1907-ID11819_20180422_1640.jpg
Could not load image: A28-HM1907-ID11819_20180423_1130.jpg
Could not load image: A28-HM1907-ID11819_20180423_1140.j

...

Could not load image: A5-HM96-ID13772_20171219_1200.jpg
Could not load image: A5-HM96-ID13772_20171214_0820.jpg
Could not load image: A5-HM96-ID13772_20171214_1220.jpg
Could not load image: A5-HM96-ID13772_20171217_0810.jpg
Could not load image: A5-HM96-ID13772_20171217_1210.jpg
Could not load image: A5-HM96-ID13772_20171215_0820.jpg
Could not load image: A5-HM96-ID13772_20171215_1220.jpg
Could not load image: A5-HM96-ID13772_20171218_0810.jpg
Could not load image: A5-HM96-ID13772_20171218_1210.jpg
Could not load image: A5-HM96-ID13772_20171221_0810.jpg
Could not load image: A5-HM96-ID13772_20171221_1210.jpg
Could not load image: A5-HM96-ID13772_20171220_1210.jpg
Could not load image: A5-HM96-ID13772_20171216_0820.jpg
Could not load image: A5-HM96-ID13772_20171216_1220.jpg
Could not load image: A5-HM96-ID13772_20171219_0810.jpg
Could not load image: A5-HM96-ID13772_20171219_1210.jpg
Could not load image: A5-HM96-ID13772_20171214_0830.jpg
Could not load image: A5-HM96-ID13772_20171214_1

Could not load image: A5-HM96-ID13772_20171223_1220.jpg
Could not load image: A5-HM96-ID13772_20171222_1430.jpg
Could not load image: A5-HM96-ID13772_20171222_0920.jpg
Could not load image: A5-HM96-ID13772_20171223_1120.jpg
Could not load image: A5-HM96-ID13772_20171222_1330.jpg
Could not load image: A5-HM96-ID13772_20171222_0820.jpg
Could not load image: A5-HM96-ID13772_20171223_1020.jpg
Could not load image: A5-HM96-ID13772_20171222_1230.jpg
Could not load image: A5-HM96-ID13772_20171223_1430.jpg
Could not load image: A5-HM96-ID13772_20171223_0920.jpg
Could not load image: A5-HM96-ID13772_20171222_1130.jpg
Could not load image: A5-HM96-ID13772_20171223_1330.jpg
Could not load image: A5-HM96-ID13772_20171224_0800.jpg
Could not load image: A5-HM96-ID13772_20171224_0810.jpg
Could not load image: A5-HM96-ID13772_20171224_0820.jpg
Could not load image: A5-HM96-ID13772_20171224_0830.jpg
Could not load image: A5-HM96-ID13772_20171224_0840.jpg
Could not load image: A5-HM96-ID13772_20171224_0

Could not load image: A5-HM96-ID13772_20180117_1100.jpg
Could not load image: A5-HM96-ID13772_20180117_1110.jpg
Could not load image: A5-HM96-ID13772_20180117_1120.jpg
Could not load image: A5-HM96-ID13772_20180117_1130.jpg
Could not load image: A5-HM96-ID13772_20180117_1140.jpg
Could not load image: A5-HM96-ID13772_20180117_1150.jpg
Could not load image: A5-HM96-ID13772_20180117_1200.jpg
Could not load image: A5-HM96-ID13772_20180117_1210.jpg
Could not load image: A5-HM96-ID13772_20180117_1220.jpg
Could not load image: A5-HM96-ID13772_20180117_1230.jpg
Could not load image: A5-HM96-ID13772_20180117_1240.jpg
Could not load image: A5-HM96-ID13772_20180117_1250.jpg
Could not load image: A5-HM96-ID13772_20180117_1300.jpg
Could not load image: A5-HM96-ID13772_20180117_1310.jpg
Could not load image: A5-HM96-ID13772_20180117_1320.jpg
Could not load image: A5-HM96-ID13772_20180117_1330.jpg
Could not load image: A5-HM96-ID13772_20180117_1340.jpg
Could not load image: A5-HM96-ID13772_20180117_1

Number of images loaded: 200500
Number of images loaded: 201000
Number of images loaded: 201500
Number of images loaded: 202000
Number of images loaded: 202500
Number of images loaded: 203000
Number of images loaded: 203500
Number of images loaded: 204000
Could not load image: A5-HM106-ID13774_20180218_1240.jpg
Number of images loaded: 204500
Number of images loaded: 205000
Could not load image: A5-HM106-ID13774_20180228_1710.jpg
Number of images loaded: 205500
Number of images loaded: 206000
Number of images loaded: 206500
Number of images loaded: 207000
Could not load image: A5-HM106-ID13774_20180430_1000.jpg
Could not load image: A5-HM106-ID13774_20180430_1010.jpg
Could not load image: A5-HM106-ID13774_20180430_1020.jpg
Could not load image: A5-HM106-ID13774_20180430_1030.jpg
Could not load image: A5-HM106-ID13774_20180430_1040.jpg
Could not load image: A5-HM106-ID13774_20180430_1050.jpg
Could not load image: A5-HM106-ID13774_20180430_1100.jpg
Could not load image: A5-HM106-ID13774_

Number of images loaded: 221500
Number of images loaded: 222000
Number of images loaded: 222500
Number of images loaded: 223000
Number of images loaded: 223500
Number of images loaded: 224000


In [17]:
highway_images = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo_train/valtestinterpolated/highway/highway_images_IDWValTest.npy')
highway_targets = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo_train/valtestinterpolated/highway/highway_targets_IDWValTest.npy')
highway_filepaths = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo_train/valtestinterpolated/highway/highway_filepaths_IDWValTest.npy')
highway_meteo = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo_train/valtestinterpolated/highway/highway_meteo_IDWValTest.npy')

In [18]:
indices = np.isnan(highway_targets)

highway_images = highway_images[~indices]
highway_targets = highway_targets[~indices]
highway_filepaths = highway_filepaths[~indices]
highway_meteo = highway_meteo[~indices]

### Ommit Labels and Change Labels Highway

#### Change

In [16]:
with open('../data/helpers/trainhighwaylabels') as file:
    for row in file:
        path, target = row.split(',')
        path_idx = np.where(highway_filepaths == path.strip("'"))
        highway_targets[path_idx] = target
    file.close()

#### Ommit

In [17]:
with open('../data/helpers/ommitancehighway') as file:
    indices = []
    for row in file:
        row = " ".join(row.split())
        path = row.strip("'")
        path_idx = np.where(highway_filepaths == row.strip("'"))
        indices.append(path_idx[0][0])
        
highway_targets = np.delete(highway_targets, indices, 0)
highway_images = np.delete(highway_images, indices, 0)
highway_filepaths = np.delete(highway_filepaths, indices, 0)
highway_meteo = np.delete(highway_meteo, indices, 0)

In [29]:
np.save('../data/processed/highway_targets_harmonie.npy', highway_targets)
np.save('../data/processed/highway_images_harmonie.npy', highway_images)
np.save('../data/processed/highway_filepaths_harmonie.npy', highway_filepaths)
np.save('../data/processed/highway_meteo_harmonie.npy', highway_meteo)

In [ ]:
test_images = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/valtestinterpolated/highway/highway_images_allIDW.npy')
highway_targets = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/valtestinterpolated/highway/highway_targets_allIDW.npy')
highway_filepaths = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/valtestinterpolated/highway/highway_filepaths_allIDW.npy')
highway_meteo = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/valtestinterpolated/highway/highway_meteo_allIDW.npy')

In [ ]:
concat = np.concatenate([meteo_df, test_meteo])
standardize_df = standardize_df.dropna(axis=0)
std_scale = preprocessing.StandardScaler().fit(standardize_df[[0, 1, 2, 3]])
highway_meteo = std_scale.transform(standardize_df[[0, 1, 2, 3]])
test_meteo = highway_meteo[-len(test_meteo):]

## Standardize Meteo Variables

In [ ]:
standardize_df = pd.DataFrame(highway_meteo, columns=[0, 1, 2, 3])
standardize_df = standardize_df.dropna(axis=0)
std_scale = preprocessing.StandardScaler().fit(standardize_df[[0, 1, 2, 3]])
highway_meteo = std_scale.transform(standardize_df[[0, 1, 2, 3]])

## Data Splits
Only highway data is split. KNMI data will be used for training as a whole and tested on the highway validation dataset.

### Validation Data
5 of the labelled highway cameras will be used for validation of both the highway train dataset and KNMI train dataset.

In [23]:
camera_list = ['A28/HM1893', 'A4/HM103', 'A5/HM86', 'A9/HM302', 'A28/HM1966']

val_idx = [i for camera in camera_list for i, v in enumerate(highway_filepaths) if camera in v]
X_validation, y_validation = highway_images[val_idx], highway_targets[val_idx].astype(int)
paths_validation, meteo_validation = highway_filepaths[val_idx], highway_meteo[val_idx]

### Highway Train Data
Highway images that are used for the validation data are deleted. What remains is the highway train data.

In [24]:
X_train_highway = np.delete(highway_images, val_idx, 0)
y_train_highway = np.delete(highway_targets, val_idx, 0).astype(int)
paths_train_highway = np.delete(highway_filepaths, val_idx, 0)
meteo_train_highway = np.delete(highway_meteo, val_idx, 0)

In [26]:
# Get the class proportions
class_counts = np.bincount(y_train_highway.astype(int))
total = len(y_train_highway)
proportion_0 = class_counts[0] / total
proportion_1 = class_counts[1] / total
proportion_2 = class_counts[2] / total

print('Class percentages:\nNo fog: {:.2f}%\nFog: {:.2f}%\nDense fog: {:.2f}%'.format(proportion_0 * 100,
                                                                              proportion_1 * 100, proportion_2 * 100))
print(class_counts)

Class percentages:
No fog: 98.51%
Fog: 0.67%
Dense fog: 0.82%
[183679   1241   1533]


## Write Numpy Arrays
Write the images, targets and filepaths to numpy arrays. These will be used for training and testing of the models.

### KNMI 

In [ ]:
np.save('../data/processed/knmi_train_images_no_change.npy', KNMI_images)
np.save('../data/processed/knmi_train_targets_no_change.npy', KNMI_targets)
np.save('../data/processed/knmi_train_filepaths_no_change.npy', KNMI_filepaths)
np.save('../data/processed/knmi_train_meteo_no_change.npy', KNMI_meteo)

### Highway Training

In [27]:
np.save('../data/processed/highway_train_images_harmonie.npy', X_train_highway)
np.save('../data/processed/highway_train_targets_harmonie.npy', y_train_highway)
np.save('../data/processed/highway_train_filepaths_harmonie.npy', paths_train_highway)
np.save('../data/processed/highway_train_meteo_harmonie.npy', meteo_train_highway)

### Highway Validation

In [28]:
np.save('../data/processed/highway_val_images_harmonie.npy', X_validation)
np.save('../data/processed/highway_val_targets_harmonie.npy', y_validation)
np.save('../data/processed/highway_val_filepaths_harmonie.npy', paths_validation)
np.save('../data/processed/highway_val_meteo_harmonie.npy', meteo_validation)

### Highway Test

In [59]:
np.save('../data/processed/test_images_harmonie.npy', test_features)
np.save('../data/processed/test_targets_harmonie.npy', test_targets)
np.save('../data/processed/test_filepaths_harmonie.npy', test_filepaths)
np.save('../data/processed/test_meteo_harmonie.npy', test_meteo)

## PCA
Optional to do a PCA for any of the datasets. It's advisable to only use a part of the no fog class data, because otherwise it's too computationally heavy.

In [ ]:
highway_images = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/testinterpolated/highway/change/highway_images.npy')
highway_targets = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/testinterpolated/highway/change/highway_targets.npy')
highway_filepaths = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/testinterpolated/highway/change/highway_filepaths.npy')
highway_meteo = np.load('/Volumes/TIMPP/UnusedKNMI/numpyfiles/meteo/testinterpolated/highway/change/highway_meteo.npy')

In [ ]:
nofog_idx = np.where(highway_targets == 0)
light_idx = np.where(highway_targets == 1)
dense_idx = np.where(highway_targets == 2)
nofog_idx = np.random.choice(nofog_idx[0], 10000)

# Select images and targets
nofog_img, nofog_target = highway_images[nofog_idx], highway_targets[nofog_idx]
light_img, light_target = highway_images[light_idx], highway_targets[light_idx]
dense_img, dense_target = highway_images[dense_idx], highway_targets[dense_idx]

In [ ]:
PCA_img = np.concatenate((nofog_img, light_img, dense_img))
PCA_targets = np.concatenate((nofog_target, light_target, dense_target))

### Main PCA

In [ ]:
from sklearn.manifold import TSNE
def run_PCA(X_data, y_data):
    '''
    Gets 2-dimensional principal components for pixel data
    
    :param X_data: Pixel values of images
    :param y_data: Corresponding target values of images
    '''
    
    # Define and transform data
    pca = PCA(n_components=2)
    principal_components= pca.fit_transform(X_data.flatten().reshape(len(X_data), IMG_SIZE * IMG_SIZE * CHANNELS))
    
    # Create DF of PCA's and targets 
    PCA_df = pd.DataFrame(data=principal_components, columns=['PCA1', 'PCA2'])
    target_df = pd.DataFrame(data=y_data, columns=['target'])
    PCA_df = pd.concat([PCA_df, target_df['target']], axis=1)
    
    return PCA_df

def run_TSNE(X_data, y_data):
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    p_c = tsne.fit_transform(X_data.flatten().reshape(len(X_data), IMG_SIZE * IMG_SIZE * CHANNELS))
    
    tsne_df = pd.DataFrame(data=p_c, columns=['PCA1', 'PCA2'])
    target_df = pd.DataFrame(data=y_data, columns=['target'])
    tsne_df = pd.concat([tsne_df, target_df['target']], axis=1)
    
    return tsne_df

def plot_PCA(PCA_df):
    '''
    This plots a PCA dataframe created with the run_PCA function
    
    :param PCA_df: PCA dataframe
    
    '''
    # Create figure
    fig = plt.figure(figsize= (8,8))
    ax = fig.add_subplot(111)
    ax.set_xlabel("PCA1")
    ax.set_ylabel("PCA2")
    targets = [0, 1, 2]
    class_names= ['Clear', 'Light Fog', 'Dense Fog']
    colors = ['r', 'g', 'b']
                                            
    for target, color in zip(targets, colors):
        indicesToKeep = PCA_df['target'] == target
        ax.scatter(PCA_df.loc[indicesToKeep, 'PCA1'],
                   PCA_df.loc[indicesToKeep, 'PCA2'],
                   c = color,
                   s = 50)

        ax.legend(class_names)
        ax.grid()

### Example PCA Analysis
PCA on a part of the highway dataset

In [ ]:
# Run PCA
PCA_df = run_TSNE(PCA_img, PCA_targets)
plot_PCA(PCA_df)